# Data Cleaning

In [2]:
import pandas as pd


## Import raw datasets

In [3]:
column_name = ['total_lot','lot_type','lots_available','carpark_number','update_date','update_time']
D1 = pd.read_csv('carpark_availability_final.csv',names=column_name)
D2 = pd.read_csv('hdb-carpark-information.csv')

In [8]:
D1.head()

,total_lot,lot_type,lots_available,carpark_number,update_date,update_time
0,105,C,18,HE12,2022-09-22,20:45:44
1,583,C,288,HLM,2022-09-22,20:46:07
2,329,C,154,RHM,2022-09-22,20:45:45
3,97,C,61,BM29,2022-09-22,20:45:54
4,96,C,52,Q81,2022-09-22,20:45:56


In [9]:
D2.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N


## Resampling raw dataset to data with 15 minute time intervals

In [10]:
D1['datetime'] = D1['update_date'] + " " + D1['update_time']
D1['datetime']= pd.to_datetime(D1['datetime'])
D1['datetime'] = D1.datetime.round('15min')

## Drop duplicate data

In [11]:
D1.drop(['update_date'],axis=1,inplace=True)
D1.drop(['update_time'],axis=1,inplace=True)
D1.drop_duplicates(inplace=True)

## Drop parking lots with less than 1854 time steps

In [12]:
for j in range(0,20):
    for i,v in D1.carpark_number.value_counts().items():
        print(i,v)
        if v < 1854:
            D1.drop(D1[D1.carpark_number == i].index, inplace=True)

U2 3875
J52 3867
B65 3866
A38 3866
HG67 3866
U23 3866
BL8 3865
TP52 3865
Y48 3864
PM13 3864
J36 3864
PM45 3863
U12 3862
BJ12 3862
HG49 3862
HG38 3862
U26 3862
B6 3861
TM31 3861
TM4 3859
Y12 3858
U57 3858
J54 3858
KB1 3858
J51 3858
J48 3858
Y49 3858
BJ3 3858
U25 3857
TP40 3857
BJ18 3857
Y10 3857
Y4 3857
CK8 3857
TP16 3856
CK58 3856
HG50 3856
U56 3856
BBM3 3856
U15 3856
BBM9 3856
S100 3856
PR4 3856
JM19 3855
JM5 3855
A63 3855
Y41 3855
W49 3855
U19 3855
Y40 3855
SE16 3855
PL46 3855
BJ4 3854
PR6 3854
A78 3854
A67 3854
B17 3854
CKM2 3854
KB10 3854
BE36 3853
BE9 3853
PR1 3853
CK74 3853
BJ42 3853
HG47 3852
TP27 3852
PR7 3852
J65 3852
U58 3852
B88 3852
T76 3852
BE4 3851
U22 3851
CKT2 3851
PM36 3851
J73 3851
TP3A 3851
Y43 3851
TP36 3850
SK15 3850
A21 3850
J45 3850
SK36 3850
SB21 3850
BE5 3850
Y14 3850
A77 3849
M25 3849
B16 3849
BJ13 3849
BE8 3849
S111 3849
B52 3849
J39 3849
KB12 3848
BJ53 3848
B30 3848
SK1 3848
A31 3848
M4 3848
T49 3848
Y6 3848
HG43 3848
CK73 3848
BJ10 3848
SI4 3848
U13 3847
HG

## Drop data with duplicate values based on carpark_number and datetime subset

In [13]:
for i,v in D1.carpark_number.value_counts().items():
    print(i,v)
    D1.drop_duplicates(subset=['carpark_number','datetime'],keep='first', inplace=True)


U2 3875
J52 3867
A38 3866
U23 3866
HG67 3866
B65 3866
TP52 3865
BL8 3865
J36 3864
PM13 3864
Y48 3864
PM45 3863
U26 3862
BJ12 3862
HG49 3862
U12 3862
B6 3861
J54 3858
J51 3858
Y49 3858
J48 3858
KB1 3858
BJ3 3858
Y4 3857
BJ18 3857
U25 3857
TP40 3857
Y10 3857
CK8 3857
S100 3856
U15 3856
TP16 3856
HG50 3856
BBM3 3856
BBM9 3856
PR4 3856
W49 3855
PL46 3855
SE16 3855
U19 3855
A63 3855
JM19 3855
B17 3854
A67 3854
KB10 3854
A78 3854
BE9 3853
BJ42 3853
CK74 3853
PR1 3853
BE36 3853
TP27 3852
B88 3852
PR7 3852
J65 3852
HG47 3852
U22 3851
PM36 3851
BE4 3851
TP3A 3851
J73 3851
SK15 3850
J45 3850
SK36 3850
BE5 3850
TP36 3850
A21 3850
SB21 3850
B16 3849
M25 3849
BE8 3849
A77 3849
B52 3849
J39 3849
BJ53 3848
T49 3848
BJ10 3848
A31 3848
KB12 3848
M4 3848
CK73 3848
B30 3848
HG43 3848
J61 3847
U13 3847
U24 3847
W36 3847
TR1 3847
Y11 3847
P6 3847
SG1 3847
J1 3847
Y8 3847
SE13 3847
PM14 3846
JM28 3846
J74 3846
HG52 3846
PR10 3846
T51 3845
J40 3845
U41 3845
TPM8 3845
Y68M 3845
CK49 3845
Y28 3845
B31 3845
U9 

## Save 1st cleaned version

In [14]:
D1.to_csv('data_droped_nov10.csv',sep=',')

## 2nd cleaning cycle started as found duplicate data

In [16]:
data = pd.read_csv('data_droped_nov10.csv')

## Drop data that has same values accross entire timestep

In [17]:
for key, _ in data.carpark_number.value_counts().items():
    print(key)
    if data[(data.carpark_number == key)].lots_available.value_counts().max() == data[(data.carpark_number == key)].shape[0]:
        data.drop(data[data.carpark_number == key].index, inplace=True)

HG62
HG52
M25
MM2
MM1
U45
U19
U23
TPM5
TP27
BJ42
BJ12
MM4
B30
KB1
TP18
TP2
TPMG
B88
U51
GBM
B16
HG10
TPM4
HG61
M3
HG50
PM14
HG67
HG49
J54
HG60
U26
B45
U25
U22
UAM1
UBKM
UBK2
U6
UA5
HG51
SE16
PP1
MNRM
M35
J48
HG30
HG89
U24T
B17
KB11
UA2
B7
KB10
U12
B53
U24
HG70
PM13
B65
PM19
PR1
BE4
CR29
TP3A
TP53
TP54
TP3
TPMF
PM45
PM41
J52
PM36
BBM2
B31
MP13
TP16
PM30
A37
B20
J37
BJ3
BBM1
MN2
PM10
BJ17
BJ18
B6
TPMD
AM20
AM18
A20
AM22
TPMJ
U38
TPMA
TPMC
MP12
PM6
J40
J36
TPMH
BE11
BE10
CC7
A70
MR5
BE14
B52
B51
B34
A71
A66
TP10
SE21
SE22
B42
B63
J39
J49
TPM7
TP30
U2
HG32
SE33
SE39
J51
SE11
SE20
TP14
TP17
J53
CR30
RCM
B75
U10
A67
BE9
J55
PM17
A68
TP67
TP40
BR14
MN1
MP1M
TR1
PM40
TP52
U65
CK73
U43
TJ41
BRBL
BRB1
B8
BJ11
CR7
PP5
HG47
SE32
P40L
B66
BE5
KB7
KB12
B70
AM79
M4
B33
A40
A9
J57
J57L
U64
B32
KB17
AM80
U66
A30
BE26
TWM4
TP8
KBM
PM29
Y48
Y48M
Y49
PM3
A60
SE40
SE24
B80
CC5
SD3
A43
A72
TP31
UA3
CR31
A25
HG95
HG96
J45
HG54
J34
BH2
JM1
J32
B35
MM3
BE42
BRM4
J41
PP9T
PM18
PM7
M16
A69
AM81
TP7
KB3
TP63
BE6


## Save 2nd cleaned version

In [18]:
data.to_csv('data_droped_nov11.csv',sep=',')

## 3rd cleaning cycle started as found duplicate data

In [19]:
data = pd.read_csv('data_droped_nov11.csv')

## Drop data with higher than single total lot value

In [20]:
for key, _ in data.carpark_number.value_counts().items():
    if len(data[data.carpark_number == key].total_lot.unique()) >= 2:
            value = data[data.carpark_number == key].total_lot.value_counts().keys()[0]
            data[data.carpark_number == key] = data[data.carpark_number == key].assign(total_lot = value)

## Drop data with higher than single lot type value

In [21]:
for key, _ in data.carpark_number.value_counts().items():
    if len(data[data.carpark_number == key].lot_type.unique()) >= 2:
            value = data[data.carpark_number == key].lot_type.value_counts().keys()[0]
            data[data.carpark_number == key] = data[data.carpark_number == key].assign(lot_type = value)

In [24]:
data['diff'] = data['total_lot'] - data['lots_available']

In [25]:
data[data['diff'] <0].carpark_number.value_counts()

BJ49    957
DWST    892
SE41    155
SB17    118
S36L     54
SDM2      1
SD2       1
Name: carpark_number, dtype: int64

## Drop data which number of available parking lots is higher than total lot number

In [26]:
data.drop(data[data.carpark_number == 'BJ49'].index, inplace=True)
data.drop(data[data.carpark_number == 'SE41'].index, inplace=True)

## Manual replacement outlier values based on EDA

In [27]:
data.loc[687855,'lots_available'] = 0
data.loc[687856,'lots_available'] = 38

## Save 3rd cleaned version

In [28]:
data.to_csv('data_droped_nov18.csv',sep=',')

## Merge time-series data with other feature dataset related to corresponding parking lots

In [4]:
data = pd.read_csv('data_droped_nov18.csv')

In [5]:
integrated_df= pd.merge(data,D2, how='left', left_on='carpark_number', right_on='car_park_no')

## Convert categorical variables to dummy variable for model training

In [6]:
cat_cols = [ "car_park_type",'short_term_parking','free_parking','night_parking','car_park_basement']

In [7]:
pd.get_dummies(integrated_df['car_park_type']) 
pd.get_dummies(integrated_df['short_term_parking'])
pd.get_dummies(integrated_df['free_parking'])
pd.get_dummies(integrated_df['night_parking'])
pd.get_dummies(integrated_df['car_park_basement'])

,N,Y
0,1,0
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
3868363,1,0
3868364,1,0
3868365,1,0
3868366,1,0


In [8]:
for i in cat_cols:
    dummy = pd.get_dummies(integrated_df[i])
    integrated_df = pd.concat([integrated_df,dummy], axis=1)

In [9]:
integrated_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,total_lot,lot_type,lots_available,carpark_number,datetime,diff,car_park_no,...,7AM-7PM,NO,WHOLE DAY,NO,SUN & PH FR 1PM-10.30PM,SUN & PH FR 7AM-10.30PM,NO,YES,N,Y
0,0,0,0,105,C,18,HE12,2022-09-22 20:45:00,87,HE12,...,0,0,1,0,0,1,0,1,1,0
1,1,1,1,583,C,288,HLM,2022-09-22 20:45:00,295,HLM,...,0,0,1,1,0,0,0,1,1,0
2,2,2,2,329,C,154,RHM,2022-09-22 20:45:00,175,RHM,...,0,0,1,0,0,1,0,1,1,0
3,3,3,3,97,C,61,BM29,2022-09-22 20:45:00,36,BM29,...,0,0,1,1,0,0,0,1,0,1
4,4,4,4,96,C,52,Q81,2022-09-22 20:45:00,44,Q81,...,0,0,1,1,0,0,0,1,1,0


## Drop unnecessary columns

In [10]:
cat_cols = ['Unnamed: 0','lot_type','car_park_no','address','type_of_parking_system', "car_park_type",'short_term_parking','free_parking','night_parking','car_park_basement']

In [11]:
integrated_df = integrated_df.drop(cat_cols, axis=1)

## Save final version

In [12]:
for index, key in enumerate(integrated_df.carpark_number.value_counts().keys()):
    print(index, key)
    integrated_df.replace(key, index, inplace=True)

0 GBM
1 TP18
2 U23
3 TP2
4 U45
5 HG62
6 HG61
7 U19
8 B30
9 M25
10 HG52
11 TPM4
12 BJ12
13 B16
14 U51
15 TPMG
16 M3
17 MM2
18 MM1
19 B88
20 MM4
21 KB1
22 HG10
23 TPM5
24 TP27
25 BJ42
26 U6
27 HG30
28 U22
29 J54
30 PR1
31 U12
32 PP1
33 B7
34 M35
35 MNRM
36 J48
37 U24
38 U24T
39 HG51
40 UBK2
41 UBKM
42 HG89
43 HG70
44 HG67
45 PM13
46 B45
47 KB11
48 KB10
49 UAM1
50 UA2
51 UA5
52 B65
53 B17
54 B53
55 U26
56 U25
57 SE16
58 PM19
59 HG49
60 HG50
61 PM14
62 HG60
63 PM36
64 CR29
65 PM6
66 TPMA
67 TPMC
68 TPMD
69 TPMF
70 TP54
71 TP16
72 TPMJ
73 PM30
74 B6
75 TP3
76 TP3A
77 TP53
78 A37
79 BJ17
80 BJ18
81 BJ3
82 BBM2
83 PM10
84 B20
85 MN2
86 MP12
87 MP13
88 BBM1
89 U38
90 J37
91 PM41
92 J52
93 B31
94 PM45
95 AM22
96 AM20
97 A20
98 AM18
99 J40
100 J36
101 BE4
102 J49
103 BE10
104 BE11
105 MN1
106 J39
107 SE11
108 SE20
109 SE21
110 SE22
111 HG32
112 TPMH
113 TP67
114 A67
115 U2
116 B75
117 U10
118 A68
119 TP10
120 TP14
121 SE33
122 SE39
123 TP17
124 A70
125 A71
126 A66
127 CC7
128 B52
129 MR5
130 BR1

In [13]:
integrated_df.to_csv('data_droped_nov18_dummy_final.csv', sep=',')